In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [14]:
!pip install imbalanced-learn

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
import pandas as pd

df = pd.read_csv('s3://ads508-20240325131610-mi4zt/training_data/')


In [7]:
df.head()

,price,freight_value,payment_installments,payment_value,review_score,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,...,order_status_canceled,order_status_delivered,order_status_invoiced,order_status_processing,order_status_shipped,order_status_unavailable,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_voucher
0,124.99,21.88,2,146.87,4,1.0,8683.0,54.0,64.0,31.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,289.00,46.48,8,335.48,5,3.0,10150.0,89.0,15.0,40.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,139.94,17.79,7,157.73,5,1.0,8267.0,52.0,52.0,17.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,149.94,23.36,1,173.30,5,1.0,12160.0,56.0,51.0,28.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,230.00,22.25,8,252.25,5,1.0,5200.0,45.0,15.0,35.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# missing value
df.isnull().sum()

price                       0
freight_value               0
payment_installments        0
payment_value               0
review_score                0
product_photos_qty          0
product_weight_g            0
product_length_cm           0
product_height_cm           0
product_width_cm            0
churn                       0
order_status_approved       0
order_status_canceled       0
order_status_delivered      0
order_status_invoiced       0
order_status_processing     0
order_status_shipped        0
order_status_unavailable    0
payment_type_boleto         0
payment_type_credit_card    0
payment_type_debit_card     0
payment_type_voucher        0
dtype: int64

In [13]:
df.dtypes

price                       float64
freight_value               float64
payment_installments          int64
payment_value               float64
review_score                  int64
product_photos_qty          float64
product_weight_g            float64
product_length_cm           float64
product_height_cm           float64
product_width_cm            float64
churn                         int64
order_status_approved       float64
order_status_canceled       float64
order_status_delivered      float64
order_status_invoiced       float64
order_status_processing     float64
order_status_shipped        float64
order_status_unavailable    float64
payment_type_boleto         float64
payment_type_credit_card    float64
payment_type_debit_card     float64
payment_type_voucher        float64
dtype: object

In [19]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X = df.drop(columns='churn')  # Features
y = df['churn']               # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=113)

smote = SMOTE(random_state=113)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [20]:
X_train_smote.to_csv("X_train_smote.csv", index=False)
y_train_smote.to_csv("y_train_smote.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

# Initialize the S3 client
s3 = boto3.client('s3')

# Specify your bucket name
bucket_name = 'ads508-20240325131610-mi4zt'

# Upload files
s3.upload_file('X_train_smote.csv', bucket_name, 'training_data/X_train_smote.csv')
s3.upload_file('y_train_smote.csv', bucket_name, 'training_data/y_train_smote.csv')
s3.upload_file('X_test.csv', bucket_name, 'training_data/X_test.csv')
s3.upload_file('y_test.csv', bucket_name, 'training_data/y_test.csv')
